In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('/content/train_bureau.xlsx')
df.head()

In [ ]:
train = pd.read_excel('/content/train_Data.xlsx')
train.head()

In [ ]:
thresh= np.quantile(train['MonthlyIncome'].fillna(0), 0.99)
train['MonthlyIncome']= train['MonthlyIncome'].apply(lambda x : np.where(x>thresh, np.nan,x))

In [ ]:
df_t = pd.read_excel('/content/test_bureau.xlsx')
df_t.head()

In [ ]:
test = pd.read_excel('/content/test_Data.xlsx')

test1= test.copy()
test.head()


In [ ]:
test['MonthlyIncome']= test['MonthlyIncome'].apply( lambda x : np.where(x > thresh, np.nan, x))


In [ ]:
l_360,l_540,l_720,l_900,l_1080,l_1260,l_1440,l_max=[],[],[],[],[],[],[],[]

for i in train['ID'].unique():
  date= pd.to_datetime(train[train['ID']==i]['DisbursalDate'].values[0])
  small_df= df[df['ID']==i]
  small_df['diff'] = small_df['DISBURSED-DT']- date
  l_360.append(small_df[small_df['diff']<= pd.Timedelta(360, unit='d')].index.values)
  l_540.append(small_df[small_df['diff']<= pd.Timedelta(540, unit='d')].index.values)
  l_720.append(small_df[small_df['diff']<= pd.Timedelta(720, unit='d')].index.values)
  l_900.append(small_df[small_df['diff']<= pd.Timedelta(900, unit='d')].index.values)
  l_1080.append(small_df[small_df['diff']<= pd.Timedelta(1080, unit='d')].index.values)
  l_1260.append(small_df[small_df['diff']<= pd.Timedelta(1260, unit='d')].index.values)
  l_1440.append(small_df[small_df['diff']<= pd.Timedelta(1440, unit='d')].index.values)
  l_max.append(small_df[small_df['diff']> pd.Timedelta(1440, unit='d')].index.values)




In [ ]:
l_360,l_540,l_720,l_900,l_1080,l_1260,l_1440,l_max
dates_train= pd.DataFrame(columns=[360,540,720,900,1080,1260,1440,'max'])

dates_train[360] = l_360
dates_train[540] = l_540
dates_train[720] = l_720
dates_train[900] = l_900
dates_train[1080] = l_1080
dates_train[1260] = l_1260
dates_train[1440] = l_1440
dates_train['max'] = l_max


In [ ]:
dates_train= pd.read_csv('dates_train.csv', index_col=0)



In [ ]:
import re
l_360= dates_train['360'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_540= dates_train['540'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_720= dates_train['720'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_900= dates_train['900'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_1080= dates_train['1080'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_1260= dates_train['1260'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_1440= dates_train['1440'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values

flat_list_360 = [int(item) for sublist in l_360 for item in sublist]
flat_list_540 = [int(item) for sublist in l_540 for item in sublist]
flat_list_720 = [int(item) for sublist in l_720 for item in sublist]
flat_list_900 = [int(item) for sublist in l_900 for item in sublist]
flat_list_1080 = [int(item) for sublist in l_1080 for item in sublist]
flat_list_1260 = [int(item) for sublist in l_1260 for item in sublist]
flat_list_1440 = [int(item) for sublist in l_1440 for item in sublist]

In [ ]:
df['CREDIT-LIMIT/SANC AMT']=df['CREDIT-LIMIT/SANC AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df['DISBURSED-AMT/HIGH CREDIT']=df['DISBURSED-AMT/HIGH CREDIT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df['INSTALLMENT-AMT']=df['INSTALLMENT-AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df['OVERDUE-AMT']=df['OVERDUE-AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df['CURRENT-BAL']=df['CURRENT-BAL'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df['WRITE-OFF-AMT']=df['WRITE-OFF-AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('float')
df['DPD - HIST']= df['DPD - HIST'].fillna(0).apply(lambda x : re.sub(r'[^0-9]','',str(x))).apply(lambda x : re.findall(r'...',str(x)))\
                .apply(lambda x: sum(list(map(lambda y: int(y), x))))

df.head()



In [ ]:
idx= df['ACCT-TYPE'].value_counts().head(3).index

df['ACCT-TYPE']= df['ACCT-TYPE'].apply(lambda x: np.where(x in idx, x, 'Other'))

idx= df['CONTRIBUTOR-TYPE'].value_counts().head(3).index

df['CONTRIBUTOR-TYPE']= df['CONTRIBUTOR-TYPE'].apply(lambda x: np.where(x in idx, x, 'Other'))


In [ ]:
df = pd.get_dummies(df, columns=['SELF-INDICATOR', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE'])
df

In [ ]:
df[['SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']]= df[['SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].astype('int')                                          

In [ ]:

df_360 = df.iloc[flat_list_360]
df_540 = df.iloc[flat_list_540]
df_720 = df.iloc[flat_list_720]
df_900 = df.iloc[flat_list_900]
df_1080 = df.iloc[flat_list_1080]
df_1440 = df.iloc[flat_list_1440]


In [ ]:
df_12month= df_360.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()


df_18month=df_540.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()

df_24month= df_720.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum() 

df_30month=df_900.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()



df_36month=df_1080.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()


                     

df_48month=df_1440.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()




df_max=df.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()  






In [ ]:
df_12month['total'] = df_12month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_12month['ACCT-TYPE_Gold Loan']+\
                df_12month['ACCT-TYPE_Other']+df_12month['ACCT-TYPE_Tractor Loan']

df_12month['gold_ratio'] = df_12month['ACCT-TYPE_Gold Loan']/df_12month['total']
df_12month['tractor_ratio'] = df_12month['ACCT-TYPE_Tractor Loan']/df_12month['total']
df_12month['nab_ratio'] = df_12month['CONTRIBUTOR-TYPE_NAB']/df_12month['total']
df_12month['nbf_ratio'] = df_12month['CONTRIBUTOR-TYPE_NBF']/df_12month['total']
df_12month['prb_ratio'] = df_12month['CONTRIBUTOR-TYPE_PRB']/df_12month['total']                       

df_18month['total'] = df_18month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_18month['ACCT-TYPE_Gold Loan']+\
                df_18month['ACCT-TYPE_Other']+df_18month['ACCT-TYPE_Tractor Loan']

df_18month['gold_ratio'] = df_18month['ACCT-TYPE_Gold Loan']/df_18month['total']
df_18month['tractor_ratio'] = df_18month['ACCT-TYPE_Tractor Loan']/df_18month['total']
df_18month['nab_ratio'] = df_18month['CONTRIBUTOR-TYPE_NAB']/df_18month['total']
df_18month['nbf_ratio'] = df_18month['CONTRIBUTOR-TYPE_NBF']/df_18month['total']
df_18month['prb_ratio'] = df_18month['CONTRIBUTOR-TYPE_PRB']/df_18month['total']


df_24month['total'] = df_24month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_24month['ACCT-TYPE_Gold Loan']+\
                df_24month['ACCT-TYPE_Other']+df_24month['ACCT-TYPE_Tractor Loan']

df_24month['gold_ratio'] = df_24month['ACCT-TYPE_Gold Loan']/df_24month['total']
df_24month['tractor_ratio'] = df_24month['ACCT-TYPE_Tractor Loan']/df_24month['total']
df_24month['nab_ratio'] = df_24month['CONTRIBUTOR-TYPE_NAB']/df_24month['total']
df_24month['nbf_ratio'] = df_24month['CONTRIBUTOR-TYPE_NBF']/df_24month['total']
df_24month['prb_ratio'] = df_24month['CONTRIBUTOR-TYPE_PRB']/df_24month['total']


df_30month['total'] = df_30month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_30month['ACCT-TYPE_Gold Loan']+\
                df_30month['ACCT-TYPE_Other']+df_30month['ACCT-TYPE_Tractor Loan']

df_30month['gold_ratio'] = df_30month['ACCT-TYPE_Gold Loan']/df_30month['total']
df_30month['tractor_ratio'] = df_30month['ACCT-TYPE_Tractor Loan']/df_30month['total']
df_30month['nab_ratio'] = df_30month['CONTRIBUTOR-TYPE_NAB']/df_30month['total']
df_30month['nbf_ratio'] = df_30month['CONTRIBUTOR-TYPE_NBF']/df_30month['total']
df_30month['prb_ratio'] = df_30month['CONTRIBUTOR-TYPE_PRB']/df_30month['total']


df_36month['total'] = df_36month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_36month['ACCT-TYPE_Gold Loan']+\
                df_36month['ACCT-TYPE_Other']+df_36month['ACCT-TYPE_Tractor Loan']

df_36month['gold_ratio'] = df_36month['ACCT-TYPE_Gold Loan']/df_36month['total']
df_36month['tractor_ratio'] = df_36month['ACCT-TYPE_Tractor Loan']/df_36month['total']
df_36month['nab_ratio'] = df_36month['CONTRIBUTOR-TYPE_NAB']/df_36month['total']
df_36month['nbf_ratio'] = df_36month['CONTRIBUTOR-TYPE_NBF']/df_36month['total']
df_36month['prb_ratio'] = df_36month['CONTRIBUTOR-TYPE_PRB']/df_36month['total']



df_48month['total'] = df_48month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_48month['ACCT-TYPE_Gold Loan']+\
                df_48month['ACCT-TYPE_Other']+df_48month['ACCT-TYPE_Tractor Loan']

df_48month['gold_ratio'] = df_48month['ACCT-TYPE_Gold Loan']/df_48month['total']
df_48month['tractor_ratio'] = df_48month['ACCT-TYPE_Tractor Loan']/df_48month['total']
df_48month['nab_ratio'] = df_48month['CONTRIBUTOR-TYPE_NAB']/df_48month['total']
df_48month['nbf_ratio'] = df_48month['CONTRIBUTOR-TYPE_NBF']/df_48month['total']
df_48month['prb_ratio'] = df_48month['CONTRIBUTOR-TYPE_PRB']/df_48month['total']


df_max['total'] = df_max['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_max['ACCT-TYPE_Gold Loan']+\
                df_max['ACCT-TYPE_Other']+df_max['ACCT-TYPE_Tractor Loan']

df_max['gold_ratio'] = df_max['ACCT-TYPE_Gold Loan']/df_max['total']
df_max['tractor_ratio'] = df_max['ACCT-TYPE_Tractor Loan']/df_max['total']
df_max['nab_ratio'] = df_max['CONTRIBUTOR-TYPE_NAB']/df_max['total']
df_max['nbf_ratio'] = df_max['CONTRIBUTOR-TYPE_NBF']/df_max['total']
df_max['prb_ratio'] = df_max['CONTRIBUTOR-TYPE_PRB']/df_max['total']



In [ ]:
train= train.join(df_12month,how='left',on='ID')
train= train.join(df_18month,how='left',on='ID',rsuffix='_18',lsuffix='_12')
train= train.join(df_24month,how='left',on='ID')
train= train.join(df_30month,how='left',on='ID',rsuffix='_30',lsuffix='_24')
train= train.join(df_36month,how='left',on='ID')
train= train.join(df_48month,how='left',on='ID',rsuffix='_48',lsuffix='_36')
train = train.join(df_max,how='left',on='ID')
train

In [ ]:
l_360t,l_540t,l_720t,l_900t,l_1080t,l_1260t,l_1440t,l_maxt=[],[],[],[],[],[],[],[]

for i in test['ID'].unique():
  date= pd.to_datetime(test[test['ID']==i]['DisbursalDate'].values[0])
  small_df= df_t[df_t['ID']==i]
  small_df['diff'] = small_df['DISBURSED-DT']- date
  l_360t.append(small_df[small_df['diff']<= pd.Timedelta(360, unit='d')].index.values)
  l_540t.append(small_df[small_df['diff']<= pd.Timedelta(540, unit='d')].index.values)
  l_720t.append(small_df[small_df['diff']<= pd.Timedelta(720, unit='d')].index.values)
  l_900t.append(small_df[small_df['diff']<= pd.Timedelta(900, unit='d')].index.values)
  l_1080t.append(small_df[small_df['diff']<= pd.Timedelta(1080, unit='d')].index.values)
  l_1260t.append(small_df[small_df['diff']<= pd.Timedelta(1260, unit='d')].index.values)
  l_1440t.append(small_df[small_df['diff']<= pd.Timedelta(1440, unit='d')].index.values)
  l_maxt.append(small_df[small_df['diff']> pd.Timedelta(1440, unit='d')].index.values)




In [ ]:
l_360,l_540,l_720,l_900,l_1080,l_1260,l_1440,l_max
dates_test= pd.DataFrame(columns=[360,540,720,900,1080,1260,1440,'max'])

dates_test[360] = l_360t
dates_test[540] = l_540t
dates_test[720] = l_720t
dates_test[900] = l_900t
dates_test[1080] = l_1080t  
dates_test[1260] = l_1260t
dates_test[1440] = l_1440t
dates_test['max'] = l_maxt


In [ ]:
dates_test= pd.read_csv('dates_test.csv')

In [ ]:
import re
l_360= dates_test['360'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_540= dates_test['540'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_720= dates_test['720'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_900= dates_test['900'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_1080= dates_test['1080'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_1260= dates_test['1260'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values
l_1440= dates_test['1440'].apply(lambda x : re.sub(r'[\[\]]','',x)).apply(lambda x: x.split()).values

flat_list_360 = [int(item) for sublist in l_360 for item in sublist]
flat_list_540 = [int(item) for sublist in l_540 for item in sublist]
flat_list_720 = [int(item) for sublist in l_720 for item in sublist]
flat_list_900 = [int(item) for sublist in l_900 for item in sublist]
flat_list_1080 = [int(item) for sublist in l_1080 for item in sublist]
flat_list_1260 = [int(item) for sublist in l_1260 for item in sublist]
flat_list_1440 = [int(item) for sublist in l_1440 for item in sublist]

In [ ]:
idx= df_t['ACCT-TYPE'].value_counts().head(3).index

df_t['ACCT-TYPE']= df_t['ACCT-TYPE'].apply(lambda x: np.where(x in idx, x, 'Other'))

idx= df_t['CONTRIBUTOR-TYPE'].value_counts().head(3).index

df_t['CONTRIBUTOR-TYPE']= df_t['CONTRIBUTOR-TYPE'].apply(lambda x: np.where(x in idx, x, 'Other'))

df_t = pd.get_dummies(df_t, columns=['SELF-INDICATOR', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE'])
df_t

In [ ]:
df_t[['SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']]= df_t[['SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].astype('int')                                          

In [ ]:
df_t['CREDIT-LIMIT/SANC AMT']=df_t['CREDIT-LIMIT/SANC AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df_t['DISBURSED-AMT/HIGH CREDIT']=df_t['DISBURSED-AMT/HIGH CREDIT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df_t['INSTALLMENT-AMT']=df_t['INSTALLMENT-AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df_t['OVERDUE-AMT']=df_t['OVERDUE-AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df_t['CURRENT-BAL']=df_t['CURRENT-BAL'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('int')
df_t['WRITE-OFF-AMT']=df_t['WRITE-OFF-AMT'].fillna(0).apply(lambda x: re.sub(r'[^0-9\-]','',str(x))).astype('float')
df_t['DPD - HIST']= df_t['DPD - HIST'].fillna(0).apply(lambda x : re.sub(r'[^0-9]','',str(x))).apply(lambda x : re.findall(r'...',str(x)))\
                .apply(lambda x: sum(list(map(lambda y: int(y), x))))
df_t360 = df_t.iloc[flat_list_360]
df_t540 = df_t.iloc[flat_list_540]
df_t720 = df_t.iloc[flat_list_720]
df_t900 = df_t.iloc[flat_list_900]
df_t1080 = df_t.iloc[flat_list_1080]
df_t1440 = df_t.iloc[flat_list_1440]


In [ ]:
df_t12month= df_t360.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()

df_t12month['total'] = df_t12month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_t12month['ACCT-TYPE_Gold Loan']+\
                df_t12month['ACCT-TYPE_Other']+df_t12month['ACCT-TYPE_Tractor Loan']

df_t12month['gold_ratio'] = df_t12month['ACCT-TYPE_Gold Loan']/df_t12month['total']
df_t12month['tractor_ratio'] = df_t12month['ACCT-TYPE_Tractor Loan']/df_t12month['total']
df_t12month['nab_ratio'] = df_t12month['CONTRIBUTOR-TYPE_NAB']/df_t12month['total']
df_t12month['nbf_ratio'] = df_t12month['CONTRIBUTOR-TYPE_NBF']/df_t12month['total']
df_t12month['prb_ratio'] = df_t12month['CONTRIBUTOR-TYPE_PRB']/df_t12month['total']                       

df_t18month=df_t540.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()
df_t18month['total'] = df_t18month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_t18month['ACCT-TYPE_Gold Loan']+\
                df_t18month['ACCT-TYPE_Other']+df_t18month['ACCT-TYPE_Tractor Loan']

df_t18month['gold_ratio'] = df_t18month['ACCT-TYPE_Gold Loan']/df_t18month['total']
df_t18month['tractor_ratio'] = df_t18month['ACCT-TYPE_Tractor Loan']/df_t18month['total']
df_t18month['nab_ratio'] = df_t18month['CONTRIBUTOR-TYPE_NAB']/df_t18month['total']
df_t18month['nbf_ratio'] = df_t18month['CONTRIBUTOR-TYPE_NBF']/df_t18month['total']
df_t18month['prb_ratio'] = df_t18month['CONTRIBUTOR-TYPE_PRB']/df_t18month['total']

df_t24month= df_t720.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()
df_t24month['total'] = df_t24month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_t24month['ACCT-TYPE_Gold Loan']+\
                df_t24month['ACCT-TYPE_Other']+df_t24month['ACCT-TYPE_Tractor Loan']

df_t24month['gold_ratio'] = df_t24month['ACCT-TYPE_Gold Loan']/df_t24month['total']
df_t24month['tractor_ratio'] = df_t24month['ACCT-TYPE_Tractor Loan']/df_t24month['total']
df_t24month['nab_ratio'] = df_t24month['CONTRIBUTOR-TYPE_NAB']/df_t24month['total']
df_t24month['nbf_ratio'] = df_t24month['CONTRIBUTOR-TYPE_NBF']/df_t24month['total']
df_t24month['prb_ratio'] = df_t24month['CONTRIBUTOR-TYPE_PRB']/df_t24month['total']

df_t30month=df_t900.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()

df_t30month['total'] = df_t30month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_t30month['ACCT-TYPE_Gold Loan']+\
                df_t30month['ACCT-TYPE_Other']+df_t30month['ACCT-TYPE_Tractor Loan']

df_t30month['gold_ratio'] = df_t30month['ACCT-TYPE_Gold Loan']/df_t30month['total']
df_t30month['tractor_ratio'] = df_t30month['ACCT-TYPE_Tractor Loan']/df_t30month['total']
df_t30month['nab_ratio'] = df_t30month['CONTRIBUTOR-TYPE_NAB']/df_t30month['total']
df_t30month['nbf_ratio'] = df_t30month['CONTRIBUTOR-TYPE_NBF']/df_t30month['total']
df_t30month['prb_ratio'] = df_t30month['CONTRIBUTOR-TYPE_PRB']/df_t30month['total']

df_t36month=df_t1080.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()


df_t36month['total'] = df_t36month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_t36month['ACCT-TYPE_Gold Loan']+\
                df_t36month['ACCT-TYPE_Other']+df_t36month['ACCT-TYPE_Tractor Loan']

df_t36month['gold_ratio'] = df_t36month['ACCT-TYPE_Gold Loan']/df_t36month['total']
df_t36month['tractor_ratio'] = df_t36month['ACCT-TYPE_Tractor Loan']/df_t36month['total']
df_t36month['nab_ratio'] = df_t36month['CONTRIBUTOR-TYPE_NAB']/df_t36month['total']
df_t36month['nbf_ratio'] = df_t36month['CONTRIBUTOR-TYPE_NBF']/df_t36month['total']
df_t36month['prb_ratio'] = df_t36month['CONTRIBUTOR-TYPE_PRB']/df_t36month['total']
                     

df_t48month=df_t1440.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()


df_t48month['total'] = df_t48month['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_t48month['ACCT-TYPE_Gold Loan']+\
                df_t48month['ACCT-TYPE_Other']+df_t48month['ACCT-TYPE_Tractor Loan']

df_t48month['gold_ratio'] = df_t48month['ACCT-TYPE_Gold Loan']/df_t48month['total']
df_t48month['tractor_ratio'] = df_t48month['ACCT-TYPE_Tractor Loan']/df_t48month['total']
df_t48month['nab_ratio'] = df_t48month['CONTRIBUTOR-TYPE_NAB']/df_t48month['total']
df_t48month['nbf_ratio'] = df_t48month['CONTRIBUTOR-TYPE_NBF']/df_t48month['total']
df_t48month['prb_ratio'] = df_t48month['CONTRIBUTOR-TYPE_PRB']/df_t48month['total']


df_tmax=df_t.groupby('ID')[['CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
        'OVERDUE-AMT', 'WRITE-OFF-AMT','DPD - HIST','SELF-INDICATOR_False', 'SELF-INDICATOR_True',
       'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
       'ACCT-TYPE_Gold Loan', 'ACCT-TYPE_Other', 'ACCT-TYPE_Tractor Loan',
       'CONTRIBUTOR-TYPE_NAB', 'CONTRIBUTOR-TYPE_NBF',
       'CONTRIBUTOR-TYPE_Other', 'CONTRIBUTOR-TYPE_PRB']].sum()  



df_tmax['total'] = df_tmax['ACCT-TYPE_Business Loan Priority Sector  Agriculture']+df_tmax['ACCT-TYPE_Gold Loan']+\
                df_tmax['ACCT-TYPE_Other']+df_tmax['ACCT-TYPE_Tractor Loan']

df_tmax['gold_ratio'] = df_tmax['ACCT-TYPE_Gold Loan']/df_tmax['total']
df_tmax['tractor_ratio'] = df_tmax['ACCT-TYPE_Tractor Loan']/df_tmax['total']
df_tmax['nab_ratio'] = df_tmax['CONTRIBUTOR-TYPE_NAB']/df_tmax['total']
df_tmax['nbf_ratio'] = df_tmax['CONTRIBUTOR-TYPE_NBF']/df_tmax['total']
df_tmax['prb_ratio'] = df_tmax['CONTRIBUTOR-TYPE_PRB']/df_tmax['total']

In [ ]:
test= test.join(df_t12month,how='left',on='ID')
test= test.join(df_t18month,how='left',on='ID',rsuffix='_18',lsuffix='_12')
test= test.join(df_t24month,how='left',on='ID')
test= test.join(df_t30month,how='left',on='ID',rsuffix='_30',lsuffix='_24')
test= test.join(df_t36month,how='left',on='ID')
test= test.join(df_t48month,how='left',on='ID',rsuffix='_48',lsuffix='_36')
test = test.join(df_tmax,how='left',on='ID')
test

In [ ]:
train['SELF-INDICATOR_False18s']= train['SELF-INDICATOR_False_18']-train['SELF-INDICATOR_False_12']
train['SELF-INDICATOR_True18s']= train['SELF-INDICATOR_True_18']-train['SELF-INDICATOR_True_12']
train['ACCT-TYPE_Business Loan Priority Sector  Agriculture18s']= train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_18']- train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_12']
train['ACCT-TYPE_Gold Loan18s']= train['ACCT-TYPE_Gold Loan_18']- train["ACCT-TYPE_Gold Loan_12"]
train['ACCT-TYPE_Other18s']= train['ACCT-TYPE_Other_18']- train["ACCT-TYPE_Other_12"]
train['ACCT-TYPE_Tractor Loan18s']= train['ACCT-TYPE_Tractor Loan_18'] - train["ACCT-TYPE_Tractor Loan_12"]
train['CONTRIBUTOR-TYPE_NAB18s']= train['CONTRIBUTOR-TYPE_NAB_18']- train['CONTRIBUTOR-TYPE_NAB_12']
train['CONTRIBUTOR-TYPE_NBF18s']= train['CONTRIBUTOR-TYPE_NBF_18']- train['CONTRIBUTOR-TYPE_NBF_12']
train['CONTRIBUTOR-TYPE_Other18s']= train['CONTRIBUTOR-TYPE_Other_18']- train['CONTRIBUTOR-TYPE_Other_12']
train['CONTRIBUTOR-TYPE_PRB18s']= train['CONTRIBUTOR-TYPE_PRB_18']- train['CONTRIBUTOR-TYPE_PRB_12']
train['SELF-INDICATOR_False24s']= train['SELF-INDICATOR_False_24']-train['SELF-INDICATOR_False_18']
train['SELF-INDICATOR_True24s']= train['SELF-INDICATOR_True_24']-train['SELF-INDICATOR_True_18']
train['ACCT-TYPE_Business Loan Priority Sector  Agriculture24s']= train["ACCT-TYPE_Business Loan Priority Sector  Agriculture_24"]- train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_18']
train['ACCT-TYPE_Gold Loan24s']= train["ACCT-TYPE_Gold Loan_24"]- train['ACCT-TYPE_Gold Loan_18']
train['ACCT-TYPE_Other24s']= train["ACCT-TYPE_Other_24"]- train['ACCT-TYPE_Other_18']
train['ACCT-TYPE_Tractor Loan24s']= train["ACCT-TYPE_Tractor Loan_24"] - train['ACCT-TYPE_Tractor Loan_18']
train['CONTRIBUTOR-TYPE_NAB24s']= train["CONTRIBUTOR-TYPE_NAB_24"]- train['CONTRIBUTOR-TYPE_NAB_18']
train['CONTRIBUTOR-TYPE_NBF24s']= train["CONTRIBUTOR-TYPE_NBF_24"]- train['CONTRIBUTOR-TYPE_NBF_18']
train['CONTRIBUTOR-TYPE_Other24s']= train["CONTRIBUTOR-TYPE_Other_24"]- train['CONTRIBUTOR-TYPE_Other_18']
train['CONTRIBUTOR-TYPE_PRB24s']= train["CONTRIBUTOR-TYPE_PRB_24"]- train['CONTRIBUTOR-TYPE_PRB_18']
train['SELF-INDICATOR_False30s']= train['SELF-INDICATOR_False_30']-train["SELF-INDICATOR_False_24"]
train['SELF-INDICATOR_True30s']= train['SELF-INDICATOR_True_30']-train["SELF-INDICATOR_True_24"]
train['ACCT-TYPE_Business Loan Priority Sector  Agriculture30s']= train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_30']- train["ACCT-TYPE_Business Loan Priority Sector  Agriculture_24"]
train['ACCT-TYPE_Gold Loan30s']= train['ACCT-TYPE_Gold Loan_30']- train["ACCT-TYPE_Gold Loan_24"]
train['ACCT-TYPE_Other30s']= train['ACCT-TYPE_Other_30']- train["ACCT-TYPE_Other_24"]
train['ACCT-TYPE_Tractor Loan30s']= train['ACCT-TYPE_Tractor Loan_30'] - train["ACCT-TYPE_Tractor Loan_24"]
train['CONTRIBUTOR-TYPE_NAB30s']= train['CONTRIBUTOR-TYPE_NAB_30']- train["CONTRIBUTOR-TYPE_NAB_24"]
train['CONTRIBUTOR-TYPE_NBF30s']= train['CONTRIBUTOR-TYPE_NBF_30']- train["CONTRIBUTOR-TYPE_NBF_24"]
train['CONTRIBUTOR-TYPE_Other30s']= train['CONTRIBUTOR-TYPE_Other_30']- train["CONTRIBUTOR-TYPE_Other_24"]
train['CONTRIBUTOR-TYPE_PRB30s']= train['CONTRIBUTOR-TYPE_PRB_30']- train["CONTRIBUTOR-TYPE_PRB_24"]

train['SELF-INDICATOR_False36s']= train["SELF-INDICATOR_False_36"]-train['SELF-INDICATOR_False_30']
train['SELF-INDICATOR_True36s']= train["SELF-INDICATOR_True_36"]-train['SELF-INDICATOR_True_30']
train['ACCT-TYPE_Business Loan Priority Sector  Agriculture36s']= train["ACCT-TYPE_Business Loan Priority Sector  Agriculture_36"]- train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_30']
train['ACCT-TYPE_Gold Loan36s']= train["ACCT-TYPE_Gold Loan_36"]- train['ACCT-TYPE_Gold Loan_30']
train['ACCT-TYPE_Other36s']= train["ACCT-TYPE_Other_36"]- train['ACCT-TYPE_Other_30']
train['ACCT-TYPE_Tractor Loan36s']= train["ACCT-TYPE_Tractor Loan_36"] - train['ACCT-TYPE_Tractor Loan_30']
train['CONTRIBUTOR-TYPE_NAB36s']= train["CONTRIBUTOR-TYPE_NAB_36"]- train['CONTRIBUTOR-TYPE_NAB_30']
train['CONTRIBUTOR-TYPE_NBF36s']= train["CONTRIBUTOR-TYPE_NBF_36"]- train['CONTRIBUTOR-TYPE_NBF_30']
train['CONTRIBUTOR-TYPE_Other36s']= train["CONTRIBUTOR-TYPE_Other_36"]- train['CONTRIBUTOR-TYPE_Other_30']
train['CONTRIBUTOR-TYPE_PRB36s']= train["CONTRIBUTOR-TYPE_PRB_36"]- train['CONTRIBUTOR-TYPE_PRB_30']

train['SELF-INDICATOR_False48s']= train['SELF-INDICATOR_False_48']-train["SELF-INDICATOR_False_36"]
train['SELF-INDICATOR_True48s']= train['SELF-INDICATOR_True_48']-train["SELF-INDICATOR_True_36"]
train['ACCT-TYPE_Business Loan Priority Sector  Agriculture48s']= train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_48']- train["ACCT-TYPE_Business Loan Priority Sector  Agriculture_36"]
train['ACCT-TYPE_Gold Loan48s']= train['ACCT-TYPE_Gold Loan_48']- train["ACCT-TYPE_Gold Loan_36"]
train['ACCT-TYPE_Other48s']= train['ACCT-TYPE_Other_48']- train["ACCT-TYPE_Other_36"]
train['ACCT-TYPE_Tractor Loan48s']= train['ACCT-TYPE_Tractor Loan_48'] - train["ACCT-TYPE_Tractor Loan_36"]
train['CONTRIBUTOR-TYPE_NAB48s']= train['CONTRIBUTOR-TYPE_NAB_48']- train["CONTRIBUTOR-TYPE_NAB_36"]
train['CONTRIBUTOR-TYPE_NBF48s']= train['CONTRIBUTOR-TYPE_NBF_48']- train["CONTRIBUTOR-TYPE_NBF_36"]
train['CONTRIBUTOR-TYPE_Other48s']= train['CONTRIBUTOR-TYPE_Other_48']- train["CONTRIBUTOR-TYPE_Other_36"]
train['CONTRIBUTOR-TYPE_PRB48s']= train['CONTRIBUTOR-TYPE_PRB_48']- train["CONTRIBUTOR-TYPE_PRB_36"]


train['SELF-INDICATOR_False>48s']= train['SELF-INDICATOR_False']-  train['SELF-INDICATOR_False_48']
train['SELF-INDICATOR_True>48s']= train['SELF-INDICATOR_True']-   train['SELF-INDICATOR_True_48']

train['ACCT-TYPE_Business Loan Priority Sector  Agriculture>48s']= train['ACCT-TYPE_Business Loan Priority Sector  Agriculture'] - train['ACCT-TYPE_Business Loan Priority Sector  Agriculture_48']
train['ACCT-TYPE_Gold Loan>48s']= train['ACCT-TYPE_Gold Loan'] - train['ACCT-TYPE_Gold Loan_48']
train['ACCT-TYPE_Other>48s']= train['ACCT-TYPE_Other'] - train['ACCT-TYPE_Other_48']
train['ACCT-TYPE_Tractor Loan>48s']= train['ACCT-TYPE_Tractor Loan'] - train['ACCT-TYPE_Tractor Loan_48']
train['CONTRIBUTOR-TYPE_NAB>48s']= train['CONTRIBUTOR-TYPE_NAB'] - train['CONTRIBUTOR-TYPE_NAB_48']
train['CONTRIBUTOR-TYPE_NBF>48s']= train['CONTRIBUTOR-TYPE_NBF'] - train['CONTRIBUTOR-TYPE_NBF_48']
train['CONTRIBUTOR-TYPE_Other>48s']= train['CONTRIBUTOR-TYPE_Other'] - train['CONTRIBUTOR-TYPE_Other_48']
train['CONTRIBUTOR-TYPE_PRB>48s']= train['CONTRIBUTOR-TYPE_PRB'] - train['CONTRIBUTOR-TYPE_PRB_48'] 
train['CREDIT-LIMIT/SANC AMT24s']=train["CREDIT-LIMIT/SANC AMT_24"] - train['CREDIT-LIMIT/SANC AMT_18']
train['DISBURSED-AMT/HIGH CREDIT24s']= train["DISBURSED-AMT/HIGH CREDIT_24"]- train['DISBURSED-AMT/HIGH CREDIT_18']
train['INSTALLMENT-AMT24s']= train["INSTALLMENT-AMT_24"]- train['INSTALLMENT-AMT_18']
train['CURRENT-BAL24s']= train["CURRENT-BAL_24"]- train['CURRENT-BAL_18']
train['OVERDUE-AMT24s']= train["OVERDUE-AMT_24"]- train['OVERDUE-AMT_18']
train['WRITE-OFF-AMT24s']= train["WRITE-OFF-AMT_24"]- train['WRITE-OFF-AMT_18']
train['DPD - HIST24s']= train["DPD - HIST_24"]- train['DPD - HIST_18']
train['CREDIT-LIMIT/SANC AMT18s']=train['CREDIT-LIMIT/SANC AMT_18'] - train["CREDIT-LIMIT/SANC AMT_12"]
train['DISBURSED-AMT/HIGH CREDIT18s']= train['DISBURSED-AMT/HIGH CREDIT_18']- train["DISBURSED-AMT/HIGH CREDIT_12"]
train['INSTALLMENT-AMT18s']= train['INSTALLMENT-AMT_18']- train["INSTALLMENT-AMT_12"]
train['CURRENT-BAL18s']= train['CURRENT-BAL_18']- train["CURRENT-BAL_12"]
train['OVERDUE-AMT18s']= train['OVERDUE-AMT_18']- train["OVERDUE-AMT_12"]
train['WRITE-OFF-AMT18s']= train['WRITE-OFF-AMT_18']- train["WRITE-OFF-AMT_12"]
train['DPD - HIST18s']= train['DPD - HIST_18']- train["DPD - HIST_12"]
train['CREDIT-LIMIT/SANC AMT30s']=train['CREDIT-LIMIT/SANC AMT_30'] - train["CREDIT-LIMIT/SANC AMT_24"]
train['DISBURSED-AMT/HIGH CREDIT30s']= train['DISBURSED-AMT/HIGH CREDIT_30']- train["DISBURSED-AMT/HIGH CREDIT_24"]
train['INSTALLMENT-AMT30s']= train['INSTALLMENT-AMT_30']- train["INSTALLMENT-AMT_24"]
train['CURRENT-BAL30s']= train['CURRENT-BAL_30']- train["CURRENT-BAL_24"]
train['OVERDUE-AMT30s']= train['OVERDUE-AMT_30']- train["OVERDUE-AMT_24"]
train['WRITE-OFF-AMT30s']= train['WRITE-OFF-AMT_30']- train["WRITE-OFF-AMT_24"]
train['DPD - HIST30s']= train['DPD - HIST_30']- train["DPD - HIST_24"]
train['CREDIT-LIMIT/SANC AMT36s']=train["CREDIT-LIMIT/SANC AMT_36"] - train['CREDIT-LIMIT/SANC AMT_30']
train['DISBURSED-AMT/HIGH CREDIT36s']= train["DISBURSED-AMT/HIGH CREDIT_36"]- train['DISBURSED-AMT/HIGH CREDIT_30']
train['INSTALLMENT-AMT36s']= train["INSTALLMENT-AMT_36"]- train['INSTALLMENT-AMT_30']
train['CURRENT-BAL36s']= train["CURRENT-BAL_36"]- train['CURRENT-BAL_30']
train['OVERDUE-AMT36s']= train["OVERDUE-AMT_36"]- train['OVERDUE-AMT_30']
train['WRITE-OFF-AMT36s']= train["WRITE-OFF-AMT_36"]- train['WRITE-OFF-AMT_30']
train['DPD - HIST36s']= train["DPD - HIST_36"]- train['DPD - HIST_30']
train['CREDIT-LIMIT/SANC AMT48s']=train['CREDIT-LIMIT/SANC AMT_48'] - train["CREDIT-LIMIT/SANC AMT_36"]
train['DISBURSED-AMT/HIGH CREDIT48s']= train['DISBURSED-AMT/HIGH CREDIT_48']- train["DISBURSED-AMT/HIGH CREDIT_36"]
train['INSTALLMENT-AMT48s']= train['INSTALLMENT-AMT_48']- train["INSTALLMENT-AMT_36"]
train['CURRENT-BAL48s']= train['CURRENT-BAL_48']- train["CURRENT-BAL_36"]
train['OVERDUE-AMT48s']= train['OVERDUE-AMT_48']- train["OVERDUE-AMT_36"]
train['WRITE-OFF-AMT48s']= train['WRITE-OFF-AMT_48']- train["WRITE-OFF-AMT_36"]
train['DPD - HIST48s']= train['DPD - HIST_48']- train["DPD - HIST_36"]
train['CREDIT-LIMIT/SANC AMT>48s']=train['CREDIT-LIMIT/SANC AMT'] - train['CREDIT-LIMIT/SANC AMT_48']
train['DISBURSED-AMT/HIGH CREDIT>48s']= train['DISBURSED-AMT/HIGH CREDIT']- train['DISBURSED-AMT/HIGH CREDIT_48']
train['INSTALLMENT-AMT>48s']= train['INSTALLMENT-AMT']- train['INSTALLMENT-AMT_48']
train['CURRENT-BAL>48s']= train['CURRENT-BAL']- train['CURRENT-BAL_48']
train['OVERDUE-AMT>48s']= train['OVERDUE-AMT']- train['OVERDUE-AMT_48']
train['WRITE-OFF-AMT>48s']= train['WRITE-OFF-AMT']- train['WRITE-OFF-AMT_48']
train['DPD - HIST>48s']= train['DPD - HIST']- train['DPD - HIST_48']

In [ ]:
train['total18s']= train['total_18'] - train['total_12']
train['total24s']= train['total_24'] - train['total_18']
train['total30s']= train['total_30'] - train['total_24']
train['total36s']= train['total_36'] - train['total_30']
train['total48s']= train['total_48'] - train['total_36']
train['total>48s']= train['total'] - train['total_48']

train['gold_ratio18s'] = train['ACCT-TYPE_Gold Loan18s']/ train['total18s']
train['gold_ratio24s'] = train['ACCT-TYPE_Gold Loan24s']/ train['total24s']
train['gold_ratio30s'] = train['ACCT-TYPE_Gold Loan30s']/ train['total30s']
train['gold_ratio36s'] = train['ACCT-TYPE_Gold Loan36s']/ train['total36s']
train['gold_ratio48s'] = train['ACCT-TYPE_Gold Loan48s']/ train['total48s']
train['gold_ratio>48s'] = train['ACCT-TYPE_Gold Loan>48s']/ train['total>48s']


In [ ]:
test['SELF-INDICATOR_False18s']= test['SELF-INDICATOR_False_18']-test['SELF-INDICATOR_False_12']
test['SELF-INDICATOR_True18s']= test['SELF-INDICATOR_True_18']-test['SELF-INDICATOR_True_12']
test['ACCT-TYPE_Business Loan Priority Sector  Agriculture18s']= test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_18']- test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_12']
test['ACCT-TYPE_Gold Loan18s']= test['ACCT-TYPE_Gold Loan_18']- test["ACCT-TYPE_Gold Loan_12"]
test['ACCT-TYPE_Other18s']= test['ACCT-TYPE_Other_18']- test["ACCT-TYPE_Other_12"]
test['ACCT-TYPE_Tractor Loan18s']= test['ACCT-TYPE_Tractor Loan_18'] - test["ACCT-TYPE_Tractor Loan_12"]
test['CONTRIBUTOR-TYPE_NAB18s']= test['CONTRIBUTOR-TYPE_NAB_18']- test['CONTRIBUTOR-TYPE_NAB_12']
test['CONTRIBUTOR-TYPE_NBF18s']= test['CONTRIBUTOR-TYPE_NBF_18']- test['CONTRIBUTOR-TYPE_NBF_12']
test['CONTRIBUTOR-TYPE_Other18s']= test['CONTRIBUTOR-TYPE_Other_18']- test['CONTRIBUTOR-TYPE_Other_12']
test['CONTRIBUTOR-TYPE_PRB18s']= test['CONTRIBUTOR-TYPE_PRB_18']- test['CONTRIBUTOR-TYPE_PRB_12']
test['SELF-INDICATOR_False24s']= test['SELF-INDICATOR_False_24']-test['SELF-INDICATOR_False_18']
test['SELF-INDICATOR_True24s']= test['SELF-INDICATOR_True_24']-test['SELF-INDICATOR_True_18']
test['ACCT-TYPE_Business Loan Priority Sector  Agriculture24s']= test["ACCT-TYPE_Business Loan Priority Sector  Agriculture_24"]- test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_18']
test['ACCT-TYPE_Gold Loan24s']= test["ACCT-TYPE_Gold Loan_24"]- test['ACCT-TYPE_Gold Loan_18']
test['ACCT-TYPE_Other24s']= test["ACCT-TYPE_Other_24"]- test['ACCT-TYPE_Other_18']
test['ACCT-TYPE_Tractor Loan24s']= test["ACCT-TYPE_Tractor Loan_24"] - test['ACCT-TYPE_Tractor Loan_18']
test['CONTRIBUTOR-TYPE_NAB24s']= test["CONTRIBUTOR-TYPE_NAB_24"]- test['CONTRIBUTOR-TYPE_NAB_18']
test['CONTRIBUTOR-TYPE_NBF24s']= test["CONTRIBUTOR-TYPE_NBF_24"]- test['CONTRIBUTOR-TYPE_NBF_18']
test['CONTRIBUTOR-TYPE_Other24s']= test["CONTRIBUTOR-TYPE_Other_24"]- test['CONTRIBUTOR-TYPE_Other_18']
test['CONTRIBUTOR-TYPE_PRB24s']= test["CONTRIBUTOR-TYPE_PRB_24"]- test['CONTRIBUTOR-TYPE_PRB_18']
test['SELF-INDICATOR_False30s']= test['SELF-INDICATOR_False_30']-test["SELF-INDICATOR_False_24"]
test['SELF-INDICATOR_True30s']= test['SELF-INDICATOR_True_30']-test["SELF-INDICATOR_True_24"]
test['ACCT-TYPE_Business Loan Priority Sector  Agriculture30s']= test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_30']- test["ACCT-TYPE_Business Loan Priority Sector  Agriculture_24"]
test['ACCT-TYPE_Gold Loan30s']= test['ACCT-TYPE_Gold Loan_30']- test["ACCT-TYPE_Gold Loan_24"]
test['ACCT-TYPE_Other30s']= test['ACCT-TYPE_Other_30']- test["ACCT-TYPE_Other_24"]
test['ACCT-TYPE_Tractor Loan30s']= test['ACCT-TYPE_Tractor Loan_30'] - test["ACCT-TYPE_Tractor Loan_24"]
test['CONTRIBUTOR-TYPE_NAB30s']= test['CONTRIBUTOR-TYPE_NAB_30']- test["CONTRIBUTOR-TYPE_NAB_24"]
test['CONTRIBUTOR-TYPE_NBF30s']= test['CONTRIBUTOR-TYPE_NBF_30']- test["CONTRIBUTOR-TYPE_NBF_24"]
test['CONTRIBUTOR-TYPE_Other30s']= test['CONTRIBUTOR-TYPE_Other_30']- test["CONTRIBUTOR-TYPE_Other_24"]
test['CONTRIBUTOR-TYPE_PRB30s']= test['CONTRIBUTOR-TYPE_PRB_30']- test["CONTRIBUTOR-TYPE_PRB_24"]

test['SELF-INDICATOR_False36s']= test["SELF-INDICATOR_False_36"]-test['SELF-INDICATOR_False_30']
test['SELF-INDICATOR_True36s']= test["SELF-INDICATOR_True_36"]-test['SELF-INDICATOR_True_30']
test['ACCT-TYPE_Business Loan Priority Sector  Agriculture36s']= test["ACCT-TYPE_Business Loan Priority Sector  Agriculture_36"]- test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_30']
test['ACCT-TYPE_Gold Loan36s']= test["ACCT-TYPE_Gold Loan_36"]- test['ACCT-TYPE_Gold Loan_30']
test['ACCT-TYPE_Other36s']= test["ACCT-TYPE_Other_36"]- test['ACCT-TYPE_Other_30']
test['ACCT-TYPE_Tractor Loan36s']= test["ACCT-TYPE_Tractor Loan_36"] - test['ACCT-TYPE_Tractor Loan_30']
test['CONTRIBUTOR-TYPE_NAB36s']= test["CONTRIBUTOR-TYPE_NAB_36"]- test['CONTRIBUTOR-TYPE_NAB_30']
test['CONTRIBUTOR-TYPE_NBF36s']= test["CONTRIBUTOR-TYPE_NBF_36"]- test['CONTRIBUTOR-TYPE_NBF_30']
test['CONTRIBUTOR-TYPE_Other36s']= test["CONTRIBUTOR-TYPE_Other_36"]- test['CONTRIBUTOR-TYPE_Other_30']
test['CONTRIBUTOR-TYPE_PRB36s']= test["CONTRIBUTOR-TYPE_PRB_36"]- test['CONTRIBUTOR-TYPE_PRB_30']

test['SELF-INDICATOR_False48s']= test['SELF-INDICATOR_False_48']-test["SELF-INDICATOR_False_36"]
test['SELF-INDICATOR_True48s']= test['SELF-INDICATOR_True_48']-test["SELF-INDICATOR_True_36"]
test['ACCT-TYPE_Business Loan Priority Sector  Agriculture48s']= test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_48']- test["ACCT-TYPE_Business Loan Priority Sector  Agriculture_36"]
test['ACCT-TYPE_Gold Loan48s']= test['ACCT-TYPE_Gold Loan_48']- test["ACCT-TYPE_Gold Loan_36"]
test['ACCT-TYPE_Other48s']= test['ACCT-TYPE_Other_48']- test["ACCT-TYPE_Other_36"]
test['ACCT-TYPE_Tractor Loan48s']= test['ACCT-TYPE_Tractor Loan_48'] - test["ACCT-TYPE_Tractor Loan_36"]
test['CONTRIBUTOR-TYPE_NAB48s']= test['CONTRIBUTOR-TYPE_NAB_48']- test["CONTRIBUTOR-TYPE_NAB_36"]
test['CONTRIBUTOR-TYPE_NBF48s']= test['CONTRIBUTOR-TYPE_NBF_48']- test["CONTRIBUTOR-TYPE_NBF_36"]
test['CONTRIBUTOR-TYPE_Other48s']= test['CONTRIBUTOR-TYPE_Other_48']- test["CONTRIBUTOR-TYPE_Other_36"]
test['CONTRIBUTOR-TYPE_PRB48s']= test['CONTRIBUTOR-TYPE_PRB_48']- test["CONTRIBUTOR-TYPE_PRB_36"]


test['SELF-INDICATOR_False>48s']= test['SELF-INDICATOR_False']-  test['SELF-INDICATOR_False_48']
test['SELF-INDICATOR_True>48s']= test['SELF-INDICATOR_True']-   test['SELF-INDICATOR_True_48']

test['ACCT-TYPE_Business Loan Priority Sector  Agriculture>48s']= test['ACCT-TYPE_Business Loan Priority Sector  Agriculture'] - test['ACCT-TYPE_Business Loan Priority Sector  Agriculture_48']
test['ACCT-TYPE_Gold Loan>48s']= test['ACCT-TYPE_Gold Loan'] - test['ACCT-TYPE_Gold Loan_48']
test['ACCT-TYPE_Other>48s']= test['ACCT-TYPE_Other'] - test['ACCT-TYPE_Other_48']
test['ACCT-TYPE_Tractor Loan>48s']= test['ACCT-TYPE_Tractor Loan'] - test['ACCT-TYPE_Tractor Loan_48']
test['CONTRIBUTOR-TYPE_NAB>48s']= test['CONTRIBUTOR-TYPE_NAB'] - test['CONTRIBUTOR-TYPE_NAB_48']
test['CONTRIBUTOR-TYPE_NBF>48s']= test['CONTRIBUTOR-TYPE_NBF'] - test['CONTRIBUTOR-TYPE_NBF_48']
test['CONTRIBUTOR-TYPE_Other>48s']= test['CONTRIBUTOR-TYPE_Other'] - test['CONTRIBUTOR-TYPE_Other_48']
test['CONTRIBUTOR-TYPE_PRB>48s']= test['CONTRIBUTOR-TYPE_PRB'] - test['CONTRIBUTOR-TYPE_PRB_48'] 
test['CREDIT-LIMIT/SANC AMT24s']=test["CREDIT-LIMIT/SANC AMT_24"] - test['CREDIT-LIMIT/SANC AMT_18']
test['DISBURSED-AMT/HIGH CREDIT24s']= test["DISBURSED-AMT/HIGH CREDIT_24"]- test['DISBURSED-AMT/HIGH CREDIT_18']
test['INSTALLMENT-AMT24s']= test["INSTALLMENT-AMT_24"]- test['INSTALLMENT-AMT_18']
test['CURRENT-BAL24s']= test["CURRENT-BAL_24"]- test['CURRENT-BAL_18']
test['OVERDUE-AMT24s']= test["OVERDUE-AMT_24"]- test['OVERDUE-AMT_18']
test['WRITE-OFF-AMT24s']= test["WRITE-OFF-AMT_24"]- test['WRITE-OFF-AMT_18']
test['DPD - HIST24s']= test["DPD - HIST_24"]- test['DPD - HIST_18']
test['CREDIT-LIMIT/SANC AMT18s']=test['CREDIT-LIMIT/SANC AMT_18'] - test["CREDIT-LIMIT/SANC AMT_12"]
test['DISBURSED-AMT/HIGH CREDIT18s']= test['DISBURSED-AMT/HIGH CREDIT_18']- test["DISBURSED-AMT/HIGH CREDIT_12"]
test['INSTALLMENT-AMT18s']= test['INSTALLMENT-AMT_18']- test["INSTALLMENT-AMT_12"]
test['CURRENT-BAL18s']= test['CURRENT-BAL_18']- test["CURRENT-BAL_12"]
test['OVERDUE-AMT18s']= test['OVERDUE-AMT_18']- test["OVERDUE-AMT_12"]
test['WRITE-OFF-AMT18s']= test['WRITE-OFF-AMT_18']- test["WRITE-OFF-AMT_12"]
test['DPD - HIST18s']= test['DPD - HIST_18']- test["DPD - HIST_12"]
test['CREDIT-LIMIT/SANC AMT30s']=test['CREDIT-LIMIT/SANC AMT_30'] - test["CREDIT-LIMIT/SANC AMT_24"]
test['DISBURSED-AMT/HIGH CREDIT30s']= test['DISBURSED-AMT/HIGH CREDIT_30']- test["DISBURSED-AMT/HIGH CREDIT_24"]
test['INSTALLMENT-AMT30s']= test['INSTALLMENT-AMT_30']- test["INSTALLMENT-AMT_24"]
test['CURRENT-BAL30s']= test['CURRENT-BAL_30']- test["CURRENT-BAL_24"]
test['OVERDUE-AMT30s']= test['OVERDUE-AMT_30']- test["OVERDUE-AMT_24"]
test['WRITE-OFF-AMT30s']= test['WRITE-OFF-AMT_30']- test["WRITE-OFF-AMT_24"]
test['DPD - HIST30s']= test['DPD - HIST_30']- test["DPD - HIST_24"]
test['CREDIT-LIMIT/SANC AMT36s']=test["CREDIT-LIMIT/SANC AMT_36"] - test['CREDIT-LIMIT/SANC AMT_30']
test['DISBURSED-AMT/HIGH CREDIT36s']= test["DISBURSED-AMT/HIGH CREDIT_36"]- test['DISBURSED-AMT/HIGH CREDIT_30']
test['INSTALLMENT-AMT36s']= test["INSTALLMENT-AMT_36"]- test['INSTALLMENT-AMT_30']
test['CURRENT-BAL36s']= test["CURRENT-BAL_36"]- test['CURRENT-BAL_30']
test['OVERDUE-AMT36s']= test["OVERDUE-AMT_36"]- test['OVERDUE-AMT_30']
test['WRITE-OFF-AMT36s']= test["WRITE-OFF-AMT_36"]- test['WRITE-OFF-AMT_30']
test['DPD - HIST36s']= test['DPD - HIST_36']- test["DPD - HIST_30"]
test['CREDIT-LIMIT/SANC AMT48s']=test['CREDIT-LIMIT/SANC AMT_48'] - test["CREDIT-LIMIT/SANC AMT_36"]
test['DISBURSED-AMT/HIGH CREDIT48s']= test['DISBURSED-AMT/HIGH CREDIT_48']- test["DISBURSED-AMT/HIGH CREDIT_36"]
test['INSTALLMENT-AMT48s']= test['INSTALLMENT-AMT_48']- test["INSTALLMENT-AMT_36"]
test['CURRENT-BAL48s']= test['CURRENT-BAL_48']- test["CURRENT-BAL_36"]
test['OVERDUE-AMT48s']= test['OVERDUE-AMT_48']- test["OVERDUE-AMT_36"]
test['WRITE-OFF-AMT48s']= test['WRITE-OFF-AMT_48']- test["WRITE-OFF-AMT_36"]
test['DPD - HIST48s']= test['DPD - HIST_48']- test["DPD - HIST_36"]
test['CREDIT-LIMIT/SANC AMT>48s']=test['CREDIT-LIMIT/SANC AMT'] - test['CREDIT-LIMIT/SANC AMT_48']
test['DISBURSED-AMT/HIGH CREDIT>48s']= test['DISBURSED-AMT/HIGH CREDIT']- test['DISBURSED-AMT/HIGH CREDIT_48']
test['INSTALLMENT-AMT>48s']= test['INSTALLMENT-AMT']- test['INSTALLMENT-AMT_48']
test['CURRENT-BAL>48s']= test['CURRENT-BAL']- test['CURRENT-BAL_48']
test['OVERDUE-AMT>48s']= test['OVERDUE-AMT']- test['OVERDUE-AMT_48']
test['WRITE-OFF-AMT>48s']= test['WRITE-OFF-AMT']- test['WRITE-OFF-AMT_48']
test['DPD - HIST>48s']= test['DPD - HIST']- test['DPD - HIST_48']

In [ ]:
test['total18s']= test['total_18'] - test['total_12']
test['total24s']= test['total_24'] - test['total_18']
test['total30s']= test['total_30'] - test['total_24']
test['total36s']= test['total_36'] - test['total_30']
test['total48s']= test['total_48'] - test['total_36']
test['total>48s']= test['total'] - test['total_48']

test['gold_ratio18s'] = test['ACCT-TYPE_Gold Loan18s']/ test['total18s']
test['gold_ratio24s'] = test['ACCT-TYPE_Gold Loan24s']/ test['total24s']
test['gold_ratio30s'] = test['ACCT-TYPE_Gold Loan30s']/ test['total30s']
test['gold_ratio36s'] = test['ACCT-TYPE_Gold Loan36s']/ test['total36s']
test['gold_ratio48s'] = test['ACCT-TYPE_Gold Loan48s']/ test['total48s']
test['gold_ratio>48s'] = test['ACCT-TYPE_Gold Loan>48s']/ test['total>48s']


In [ ]:
X= train.drop(['ID','Area','City','DisbursalDate','MaturityDAte','AuthDate','AssetID','SupplierID','AGE','Top-up Month','State',
               'CREDIT-LIMIT/SANC AMT_18',
 'DISBURSED-AMT/HIGH CREDIT_18',
 'INSTALLMENT-AMT_18',
 'CURRENT-BAL_18',
 'OVERDUE-AMT_18',
 'WRITE-OFF-AMT_18',
 'DPD - HIST_18',
 'SELF-INDICATOR_False_18',
 'SELF-INDICATOR_True_18',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_18',
 'ACCT-TYPE_Gold Loan_18',
 'ACCT-TYPE_Other_18',
 'ACCT-TYPE_Tractor Loan_18',
 'CONTRIBUTOR-TYPE_NAB_18',
 'CONTRIBUTOR-TYPE_NBF_18',
 'CONTRIBUTOR-TYPE_Other_18',
 'CONTRIBUTOR-TYPE_PRB_18',
 'total_18',
 'gold_ratio_18',
 'tractor_ratio_18',
 'nab_ratio_18',
 'nbf_ratio_18',
 'prb_ratio_18',
 'CREDIT-LIMIT/SANC AMT_24',
 'DISBURSED-AMT/HIGH CREDIT_24',
 'INSTALLMENT-AMT_24',
 'CURRENT-BAL_24',
 'OVERDUE-AMT_24',
 'WRITE-OFF-AMT_24',
 'DPD - HIST_24',
 'SELF-INDICATOR_False_24',
 'SELF-INDICATOR_True_24',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_24',
 'ACCT-TYPE_Gold Loan_24',
 'ACCT-TYPE_Other_24',
 'ACCT-TYPE_Tractor Loan_24',
 'CONTRIBUTOR-TYPE_NAB_24',
 'CONTRIBUTOR-TYPE_NBF_24',
 'CONTRIBUTOR-TYPE_Other_24',
 'CONTRIBUTOR-TYPE_PRB_24',
 'total_24',
 'gold_ratio_24',
 'tractor_ratio_24',
 'nab_ratio_24',
 'nbf_ratio_24',
 'prb_ratio_24',
 'CREDIT-LIMIT/SANC AMT_30',
 'DISBURSED-AMT/HIGH CREDIT_30',
 'INSTALLMENT-AMT_30',
 'CURRENT-BAL_30',
 'OVERDUE-AMT_30',
 'WRITE-OFF-AMT_30',
 'DPD - HIST_30',
 'SELF-INDICATOR_False_30',
 'SELF-INDICATOR_True_30',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_30',
 'ACCT-TYPE_Gold Loan_30',
 'ACCT-TYPE_Other_30',
 'ACCT-TYPE_Tractor Loan_30',
 'CONTRIBUTOR-TYPE_NAB_30',
 'CONTRIBUTOR-TYPE_NBF_30',
 'CONTRIBUTOR-TYPE_Other_30',
 'CONTRIBUTOR-TYPE_PRB_30',
 'total_30',
 'gold_ratio_30',
 'tractor_ratio_30',
 'nab_ratio_30',
 'nbf_ratio_30',
 'prb_ratio_30',
 'CREDIT-LIMIT/SANC AMT_36',
 'DISBURSED-AMT/HIGH CREDIT_36',
 'INSTALLMENT-AMT_36',
 'CURRENT-BAL_36',
 'OVERDUE-AMT_36',
 'WRITE-OFF-AMT_36',
 'DPD - HIST_36',
 'SELF-INDICATOR_False_36',
 'SELF-INDICATOR_True_36',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_36',
 'ACCT-TYPE_Gold Loan_36',
 'ACCT-TYPE_Other_36',
 'ACCT-TYPE_Tractor Loan_36',
 'CONTRIBUTOR-TYPE_NAB_36',
 'CONTRIBUTOR-TYPE_NBF_36',
 'CONTRIBUTOR-TYPE_Other_36',
 'CONTRIBUTOR-TYPE_PRB_36',
 'total_36',
 'gold_ratio_36',
 'tractor_ratio_36',
 'nab_ratio_36',
 'nbf_ratio_36',
 'prb_ratio_36',
 'CREDIT-LIMIT/SANC AMT_48',
 'DISBURSED-AMT/HIGH CREDIT_48',
 'INSTALLMENT-AMT_48',
 'CURRENT-BAL_48',
 'OVERDUE-AMT_48',
 'WRITE-OFF-AMT_48',
 'DPD - HIST_48',
 'SELF-INDICATOR_False_48',
 'SELF-INDICATOR_True_48',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_48',
 'ACCT-TYPE_Gold Loan_48',
 'ACCT-TYPE_Other_48',
 'ACCT-TYPE_Tractor Loan_48',
 'CONTRIBUTOR-TYPE_NAB_48',
 'CONTRIBUTOR-TYPE_NBF_48',
 'CONTRIBUTOR-TYPE_Other_48',
 'CONTRIBUTOR-TYPE_PRB_48',
 'total_48',
 'gold_ratio_48',
 'tractor_ratio_48',
 'nab_ratio_48',
 'nbf_ratio_48',
 'prb_ratio_48',
 'CREDIT-LIMIT/SANC AMT',
 'DISBURSED-AMT/HIGH CREDIT',
 'INSTALLMENT-AMT',
 'CURRENT-BAL',
 'OVERDUE-AMT',
 'WRITE-OFF-AMT',
 'DPD - HIST',
 'SELF-INDICATOR_False',
 'SELF-INDICATOR_True',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
 'ACCT-TYPE_Gold Loan',
 'ACCT-TYPE_Other',
 'ACCT-TYPE_Tractor Loan',
 'CONTRIBUTOR-TYPE_NAB',
 'CONTRIBUTOR-TYPE_NBF',
 'CONTRIBUTOR-TYPE_Other',
 'CONTRIBUTOR-TYPE_PRB',
 'total',
 'gold_ratio',
 'tractor_ratio',
 'nab_ratio',
 'nbf_ratio',
 'prb_ratio'
 ]
 , axis=1)
y = train['Top-up Month']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y)

In [ ]:
X_train['PaymentMode']= X_train['PaymentMode'].apply(lambda x: np.where(x=='Cheque','PDC',x))
train_fin= X_train.drop(['Frequency','InstlmentMode','ZiPCODE','ManufacturerID', 'BranchID','SEX'], axis=1)
from sklearn.impute import KNNImputer
imputer = KNNImputer()
imputer.fit(train_fin[['AmountFinance','EMI','MonthlyIncome']])
two_cols= imputer.transform(train_fin[['AmountFinance','EMI','MonthlyIncome']])
train_fin['MonthlyIncome']= two_cols[:,1]
X_train = pd.get_dummies(train_fin, columns=['LoanStatus'\
                               ,'PaymentMode',])


In [ ]:
X_val['PaymentMode']= X_val['PaymentMode'].apply(lambda x: np.where(x=='Cheque','PDC',x))

val_fin= X_val.drop(['Frequency','InstlmentMode','SEX','ZiPCODE','ManufacturerID', 'BranchID'], axis=1)

two_cols= imputer.transform(val_fin[['AmountFinance','EMI','MonthlyIncome']])
val_fin['MonthlyIncome']= two_cols[:,1]


In [ ]:
X_valid = pd.get_dummies(val_fin, columns=['LoanStatus'\
                               ,'PaymentMode'])
X_valid

In [ ]:
for col in X_train:
  if col not in X_valid:
    print(col)

In [ ]:
X_train.fillna(0, inplace=True)

In [ ]:
X_valid.fillna(0, inplace=True)

In [ ]:

X_valid = X_valid[X_train.columns]


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
est = [('ada', AdaBoostClassifier(n_estimators=300)),('lgbm', LGBMClassifier(n_estimators=300)),('xgb',XGBClassifier(n_estimators=300))]

model = StackingClassifier(estimators=est, verbose=3)

model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 14.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 35.9min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logist

StackingClassifier(cv=None,
                   estimators=[('ada',
                                AdaBoostClassifier(algorithm='SAMME.R',
                                                   base_estimator=None,
                                                   learning_rate=1.0,
                                                   n_estimators=300,
                                                   random_state=None)),
                               ('lgbm',
                                LGBMClassifier(boosting_type='gbdt',
                                               class_weight=None,
                                               colsample_bytree=1.0,
                                               importance_type='split',
                                               learning_rate=0.1, max_depth=-1,
                                               min_child_samples=20,
                                               min_child_weight=0.001,
                                          

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import accuracy_score
y_pred= model.predict(X_valid)
print(accuracy_score(y_val, y_pred))
print(f1_score(y_val, y_pred, average='macro'))

0.8939295800191724
0.6347089330970758


In [ ]:
test= test.drop(['ID','Area','City','DisbursalDate','MaturityDAte','AuthDate','AssetID','SupplierID','AGE','State',
               'CREDIT-LIMIT/SANC AMT_18',
 'DISBURSED-AMT/HIGH CREDIT_18',
 'INSTALLMENT-AMT_18',
 'CURRENT-BAL_18',
 'OVERDUE-AMT_18',
 'WRITE-OFF-AMT_18',
 'DPD - HIST_18',
 'SELF-INDICATOR_False_18',
 'SELF-INDICATOR_True_18',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_18',
 'ACCT-TYPE_Gold Loan_18',
 'ACCT-TYPE_Other_18',
 'ACCT-TYPE_Tractor Loan_18',
 'CONTRIBUTOR-TYPE_NAB_18',
 'CONTRIBUTOR-TYPE_NBF_18',
 'CONTRIBUTOR-TYPE_Other_18',
 'CONTRIBUTOR-TYPE_PRB_18',
 'total_18',
 'gold_ratio_18',
 'tractor_ratio_18',
 'nab_ratio_18',
 'nbf_ratio_18',
 'prb_ratio_18',
 'CREDIT-LIMIT/SANC AMT_24',
 'DISBURSED-AMT/HIGH CREDIT_24',
 'INSTALLMENT-AMT_24',
 'CURRENT-BAL_24',
 'OVERDUE-AMT_24',
 'WRITE-OFF-AMT_24',
 'DPD - HIST_24',
 'SELF-INDICATOR_False_24',
 'SELF-INDICATOR_True_24',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_24',
 'ACCT-TYPE_Gold Loan_24',
 'ACCT-TYPE_Other_24',
 'ACCT-TYPE_Tractor Loan_24',
 'CONTRIBUTOR-TYPE_NAB_24',
 'CONTRIBUTOR-TYPE_NBF_24',
 'CONTRIBUTOR-TYPE_Other_24',
 'CONTRIBUTOR-TYPE_PRB_24',
 'total_24',
 'gold_ratio_24',
 'tractor_ratio_24',
 'nab_ratio_24',
 'nbf_ratio_24',
 'prb_ratio_24',
 'CREDIT-LIMIT/SANC AMT_30',
 'DISBURSED-AMT/HIGH CREDIT_30',
 'INSTALLMENT-AMT_30',
 'CURRENT-BAL_30',
 'OVERDUE-AMT_30',
 'WRITE-OFF-AMT_30',
 'DPD - HIST_30',
 'SELF-INDICATOR_False_30',
 'SELF-INDICATOR_True_30',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_30',
 'ACCT-TYPE_Gold Loan_30',
 'ACCT-TYPE_Other_30',
 'ACCT-TYPE_Tractor Loan_30',
 'CONTRIBUTOR-TYPE_NAB_30',
 'CONTRIBUTOR-TYPE_NBF_30',
 'CONTRIBUTOR-TYPE_Other_30',
 'CONTRIBUTOR-TYPE_PRB_30',
 'total_30',
 'gold_ratio_30',
 'tractor_ratio_30',
 'nab_ratio_30',
 'nbf_ratio_30',
 'prb_ratio_30',
 'CREDIT-LIMIT/SANC AMT_36',
 'DISBURSED-AMT/HIGH CREDIT_36',
 'INSTALLMENT-AMT_36',
 'CURRENT-BAL_36',
 'OVERDUE-AMT_36',
 'WRITE-OFF-AMT_36',
 'DPD - HIST_36',
 'SELF-INDICATOR_False_36',
 'SELF-INDICATOR_True_36',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_36',
 'ACCT-TYPE_Gold Loan_36',
 'ACCT-TYPE_Other_36',
 'ACCT-TYPE_Tractor Loan_36',
 'CONTRIBUTOR-TYPE_NAB_36',
 'CONTRIBUTOR-TYPE_NBF_36',
 'CONTRIBUTOR-TYPE_Other_36',
 'CONTRIBUTOR-TYPE_PRB_36',
 'total_36',
 'gold_ratio_36',
 'tractor_ratio_36',
 'nab_ratio_36',
 'nbf_ratio_36',
 'prb_ratio_36',
 'CREDIT-LIMIT/SANC AMT_48',
 'DISBURSED-AMT/HIGH CREDIT_48',
 'INSTALLMENT-AMT_48',
 'CURRENT-BAL_48',
 'OVERDUE-AMT_48',
 'WRITE-OFF-AMT_48',
 'DPD - HIST_48',
 'SELF-INDICATOR_False_48',
 'SELF-INDICATOR_True_48',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture_48',
 'ACCT-TYPE_Gold Loan_48',
 'ACCT-TYPE_Other_48',
 'ACCT-TYPE_Tractor Loan_48',
 'CONTRIBUTOR-TYPE_NAB_48',
 'CONTRIBUTOR-TYPE_NBF_48',
 'CONTRIBUTOR-TYPE_Other_48',
 'CONTRIBUTOR-TYPE_PRB_48',
 'total_48',
 'gold_ratio_48',
 'tractor_ratio_48',
 'nab_ratio_48',
 'nbf_ratio_48',
 'prb_ratio_48',
 'CREDIT-LIMIT/SANC AMT',
 'DISBURSED-AMT/HIGH CREDIT',
 'INSTALLMENT-AMT',
 'CURRENT-BAL',
 'OVERDUE-AMT',
 'WRITE-OFF-AMT',
 'DPD - HIST',
 'SELF-INDICATOR_False',
 'SELF-INDICATOR_True',
 'ACCT-TYPE_Business Loan Priority Sector  Agriculture',
 'ACCT-TYPE_Gold Loan',
 'ACCT-TYPE_Other',
 'ACCT-TYPE_Tractor Loan',
 'CONTRIBUTOR-TYPE_NAB',
 'CONTRIBUTOR-TYPE_NBF',
 'CONTRIBUTOR-TYPE_Other',
 'CONTRIBUTOR-TYPE_PRB',
 'total',
 'gold_ratio',
 'tractor_ratio',
 'nab_ratio',
 'nbf_ratio',
 'prb_ratio'], axis=1)



test['PaymentMode']= test['PaymentMode'].apply(lambda x: np.where(x=='Cheque','PDC',x))
test_fin= test.drop(['Frequency','InstlmentMode','SEX','ZiPCODE','ManufacturerID', 'BranchID'], axis=1)
two_cols= imputer.transform(test_fin[['EMI','MonthlyIncome']])
test_fin['MonthlyIncome']= two_cols[:,1]


In [ ]:
X_test = pd.get_dummies(test_fin, columns=['LoanStatus'\
                               ,'PaymentMode'])
X_test

In [ ]:
for col in X_train:
  if col not in X_test:
    X_test[col]=0

In [ ]:
X_test = X_test[X_train.columns]

In [ ]:
X_test.fillna(0, inplace=True)

In [ ]:
y_pred = model.predict(X_test)

df_fin=pd.DataFrame({'ID':test.ID, 'Top-up Month':y_pred})
df_fin

,ID,Top-up Month
0,4,No Top-up Service
1,5,> 48 Months
2,6,No Top-up Service
3,25,No Top-up Service
4,119,No Top-up Service
5,126,No Top-up Service
6,127,> 48 Months
7,345,No Top-up Service
8,361,No Top-up Service
9,393,No Top-up Service


In [ ]:
df_fin['Top-up Month'].value_counts()

No Top-up Service    13822
 > 48 Months           429
36-48 Months           169
30-36 Months            87
18-24 Months            85
24-30 Months            81
12-18 Months            72
Name: Top-up Month, dtype: int64

In [ ]:
df_fin.to_csv('XGB8.csv', index=None)

from google.colab import files

files.download('XGB8.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>